In [1]:
library(qpdf)    # Para manipular PDFs
library(glue)    # Para construir strings fácilmente (opcional)

# Crear un PDF con 5 páginas numeradas
create_sample_pdf <- function(file) {
  pdf(file, width = 8.5, height = 11)  # tamaño carta
  for (page_num in 1:5) {
    plot.new()
    text(0.5, 0.5, labels = paste("Page", page_num), cex = 3)
  }
  dev.off()
}

# Guardar archivo con prefijo de fecha
save_pdf_with_date <- function(input_file, suffix_name) {
  date_prefix <- format(Sys.Date(), "%Y%m%d_")
  output_file <- paste0(date_prefix, suffix_name)
  file.copy(input_file, output_file, overwrite = TRUE)
  message("Saved file: ", output_file)
  return(output_file)
}

# Reordenar páginas de un PDF dado un vector con el nuevo orden
reorder_pdf_pages <- function(input_file, new_order, output_file) {
  # qpdf espera que las páginas empiecen en 1, no en 0 (como en Python)
  new_order_plus1 <- new_order + 1
  qpdf::pdf_subset(input_file, pages = new_order_plus1, output = output_file)
  return(output_file)
}

# ----------- Uso -------------

# Crear PDF temporal
temp_pdf <- tempfile(fileext = ".pdf")
create_sample_pdf(temp_pdf)

# Guardar con fecha y nombre original
file_original <- save_pdf_with_date(temp_pdf, "20250619_original_sample.pdf")

# Definir nuevo orden (como en Python: [4,0,1,2,3])
new_order <- c(4, 0, 1, 2, 3)

# Reordenar PDF y guardar
temp_reordered_pdf <- tempfile(fileext = ".pdf")
file_reordered <- reorder_pdf_pages(file_original, new_order, temp_reordered_pdf)
file_reordered <- save_pdf_with_date(file_reordered, "20250619_reordered_sample.pdf")


Warning message:
"package 'qpdf' was built under R version 4.4.3"


pdf 
  2

Saved file: 20250619_original_sample.pdf

Saved file: 20250619_reordered_sample.pdf

